### Семинар 7. Умные указатели (C++11)

https://en.cppreference.com/w/cpp/memory/unique_ptr

https://en.cppreference.com/w/cpp/memory/shared_ptr

https://en.cppreference.com/w/cpp/memory/weak_ptr

https://en.cppreference.com/w/cpp/memory/unique_ptr/make_unique

https://en.cppreference.com/w/cpp/memory/shared_ptr/make_shared

<br />

##### Мотивацонные примеры, зачем нужны умные указатели:

Вариант 1. Мы пишем своих angry birds. Нужно уметь подгружать для игрушки ресурсы по сети.

In [ ]:
class NetworkSystem
{
public:
    virtual ~NetworkSystem() = default;

    virtual std::string load(const std::string& url) = 0;
};

class GameResourcesLoader
{
public:
    GameResourcesLoader(NetworkSystem* network) {...}

private:
    NetworkSystem* network_;
};

class PromoEventsLoader
{
public:
    PromoEventsLoader(NetworkSystem* network) {...}

private:
    NetworkSystem* network_;
};

В чём проблема?

<br />

Вариант 2. Мы реализуем первое домашнее задание и решили повыпендриваться с интерфейсами хешеров:

In [ ]:
class Hasher
{
public:
    virtual ~Hasher() = default;

    virtual std::uint32_t calc_hash(std::istream& is) = 0;
};

Hasher *create_hasher(HasherType type)
{
    if (type == CRC32)
        return new HasherCRC32();
    else
        return new HasherSUM32();
}

bool run(const std::string& filename, HasherType type)
{
    std::ifstream ifs(filename);
    if (!ifs)
        return false;

    Hasher *hasher = create_hasher(type);

    std::cout << hasher->calc_hash(ifs);

    delete hasher;
}

В чём проблема?

<br />

Вариант 3. Посложнее. В чём проблема здесь?

In [ ]:
class VideoDecoder
{
public:
    VideoDecoder()
        : codecs_{
            new VideoCodecDIVX(),
            new VideoCodecX264(),
            new VideoCodecXVID()
        }
    {}

    ~VideoDecoder() {
        for (auto* codec: codecs_)
            delete codec;
    }
    
    ...;

private:
    std::vector<VideoCodec*> codecs_;
};

<br />

##### Варианты умных (и не очень) указателей

Эти правила появились с С++1, и отчасти являются следствием реализации умных указетелей, отчасти - договорённости. Те места, где правила нарушаются, должны быть в коде хорошо откомментированы.

* `unique_ptr` - указатель, описывающий уникальное владение объектом
* `shared_ptr` - указатель, описывающий множественные владние объектом
* `weak_ptr` - указатель для отслеживания времени жизни shared-объектов
* `raw pointer` - невладеющий указатель

__Замечание__: за исключением задокументированных случаев сырой указатель - невладеющий, код, работающий с сырым указателем, не должен вызывать `free` и `delete`. Т.е.:

* метод возвращает невладеющий указатель, ответственность за удаление объекта лежит на классе

```
class ItemsStorage
{
    ...
    Item* find(int key) const;
};
```

* функция возвращает невладеющий указатель, она знает, что вызывающий код не должен управлять временем жизни объекта

```
const char* get_name()
{
    return "Dobryna Nikitich";
}
```

* функция принимает невладеющий указатель, она не в праве управлять временем жизни объекта. Семантика указателя: объект может отсутствовать.

```
Home build_home(const Roof* roof);
```

* функция принимает невладеющий указатель и возвращает невлащеющий указатель:

```
const char* find_char(const char* str, char c);
```

* особый случай - функции `malloc` && `free`:

```
void* malloc(size_t size);
free(void *);
```

<br />

##### std::unique_ptr IR

Упрощённый вариант для демонстрации

In [ ]:
template<typename T>
class unique_ptr
{
public:
    unique_ptr() : ptr(nullptr) {}
    
    unique_ptr(T* p) : ptr(p) {}
    
    ~unique_ptr()
    {
        delete ptr;
    }
    
    // # указатель уникальный - копировать его нельзя
    unique_ptr(const unique_ptr&) = delete;
    unique_ptr& operator = (const unique_ptr&) = delete;
    
    // # а перемещать из одного объекта в другой - можно:
    unique_ptr(unique_ptr&& rhs) noexcept
    {
        ptr = rhs.ptr;
        rhs.ptr = nullptr;  // # 1. ~unique_ptr отработает нормально?
                            // # 2. nullptr - специфицирован!
    }
    unique_ptr& operator = (unique_ptr&& rhs) noexcept
    {
        if (this != &rhs)
        {
            ptr = rhs.ptr;
            rhs.ptr = nullptr;  
        }
        return *this;
    }
    
    T& operator ->() const noexcept
    {
        return *ptr;
    }    

private:
    T* ptr;
};

Чему равен `sizeof` такого `unique_ptr` ?

<br />

#### std::make_unique (C++14)

In [ ]:
class VideoDecoder
{
public:
    VideoDecoder()
        : codecs_{
            new VideoCodecDIVX(),
            new VideoCodecX264(),
            new VideoCodecXVID()
        }
    {}

    ~VideoDecoder() {
        for (auto* codec: codecs_)
            delete codec;
    }
    
    ...;

private:
    std::vector<VideoCodec*> codecs_;
};

In [ ]:
class VideoDecoder
{
public:
    VideoDecoder()
        : codecs_{
            std::make_unique<VideoCodecDIVX>(),
            std::make_unique<VideoCodecX264>(),
            std::make_unique<VideoCodecXVID>()
        }
    {}

    ~VideoDecoder() = default;
    
    ...;

private:
    std::vector<std::unique_ptr<VideoCodec>> codecs_;
};

<br />

##### std::shared_ptr IR

Перед тем как показать ответ, попробуйте ответить на следующие вопросы:

Объект должен умереть, тогда и только тогда когда умрёт __последний__ его владелец. Какую информацию придётся хранить?

Где эту информацию хранить? Как насчёт такого примера:

```
std::shared_ptr<T> a(new T);
std::shared_ptr<T> b = a;
std::shared_ptr<T> c = b;

a.reset();
b.reset();

// объект должен жить, т.к. им владеют через c

```

<br />

Ответ:

![](sp_internals.PNG)

<br />

##### Как работает shared_ptr

Потренируемся на кошках. Эти примеры надо рисовать на доске (что происходит в памяти), так понимают лучше

In [ ]:
{
    std::shared_ptr<Cat> cat1(new Cat);

    std::shared_ptr<Cat> cat2;

    cat2 = cat1;

    std::shared_ptr<Cat> cat3 = cat1;

    cat1.reset();

    std::weak_ptr<Cat> weak_cat = cat2;

    std::shared_ptr<Cat> cat4 = weak_cat.lock();
    if (cat4)
        ...;
}

<br />

##### Специфические ошибки с shared_ptr

* Один объект - несколько кластеров `shared_ptr`-ов

    ```
    Cat* y = new Cat();
    std::shared_ptr<Cat> cat1(y);
    std::shared_ptr<Cat> cat2(y);
    ```

* циклы владения

```
class Cat
{
public:
    void add_frient(std::shared_ptr<Cat> f)
    {
        friends.push_back(std::move(f));
    }

private:
    std::vetor<std::shared_ptr<Cat>> friends;
};

{
    auto barsic = std::make_shared<Cat>();
    auto marsic = std::make_shared<Cat>();
    barsic->add_friend(marsic);
    marsic->add_friend(barsic);
}
```

<br />

##### make_shared - способ быстрее создавать shared_ptr

https://arne-mertz.de/2018/09/make_shared-vs-the-normal-shared_ptr-constructor/

In [ ]:
std::vector<std::shared_ptr<Cat>> herd;
herd.reserve(100);
for (int i = 0; i < 100; ++i)
    herd.push_back(std::shared_ptr<Cat>(new Cat()));

![](shared_ptr_naive.png)

200 аллокаций на объекты + 1 аллокация на вектор

<br />

In [ ]:
std::vector<std::shared_ptr<Cat>> herd;
herd.reserve(100);
for (int i = 0; i < 100; ++i)
    herd.push_back(std::make_shared_ptr<Cat>());

![](shared_ptr_make_shared.png)

100 аллокаций на объекты + 1 аллокация на вектор

<br />

#### Правила работы с уникальными указателями:

* Владение объектом:
  * по значению
  * через `std::unique_ptr` в случае единоличного владения
  * через `std::shared_ptr` в случае множественного владения
  
  Не надо так!
  ```
  struct Point
  {
      std::unique_ptr<float> x;
      std::unique_ptr<float> y;
  };
  ```
  
  Будьте человечнее:
  ```
  struct Point
  {
      float x;
      float y;
  };
  ```
  
* В нормальном коде почти никогда не нужно явно вызывать `new`/`delete`. Для этого есть `std::make_unique` и `std::make_shared`:
  * `std::make_unique` помогает избежать проблем с утечкой при исключениях
  * `std::make_shared` дополнительно работает быстрее, т.к. выполняет меньше аллокаций

* Для разрыва циклов используйте `std::weak_ptr`

* Не надо без нужды передавать в функцию умный указатель - передавайте сам объект. Умный указатель передаётся только в том случае, если функция работает с методом владения объектом:

    Случай, когда функции не должны работать со способом владения объектом:
    ```
    void print_person(const Person& person)
    {
        std::cout << person.name << ' ' << person.surname << ' ' << person.age;
    }

    void make_elder(Person& person)
    {
        ++person.age;

        if (person.age == 33)
            std::cout << "time to fight!" << std::endl;
    }

    std::shared_ptr<Person> ilya = std::make_shared<Person>("Ilya", "Muromec", 32);
    print_person(*ilya);
    make_elder(*ilya);
    ```
    
    Случай, когда должны:
    ```
    class Squad
    {
    public:
        void add_warrior(std::shared_ptr<Person> warrior)
        {
            warriors_.push_back(warrior);
            
            // лучше: warriors_.push_back(std::move(warrior));
            // почему лучше?            
        }
        
    private:
        std::vector<std::shared_ptr<Person>> warriors_;
    };
    
    Squad s;
    std::shared_ptr<Person> ilya = std::make_shared<Person>("Ilya", "Muromec", 32);
    s.add_warrior(ilya);
    ```

<br />

За пределами семинара:
    
* custom deleter
* custom allocator
* scope exit function implementation with `std::unique_ptr`
* `shared_from_this`
* `std::shared_ptr` to class member
* intrusive pointers